## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-16-16-51-38 +0000,nypost,Nestlé chairman Paul Bulcke steps down after C...,https://nypost.com/2025/09/16/business/nestle-...
1,2025-09-16-16-47-32 +0000,nypost,Famed children’s book author Robert Munsch rev...,https://nypost.com/2025/09/16/world-news/kids-...
2,2025-09-16-16-46-30 +0000,nyt,"Johnson Releases Spending Bill, Daring Democra...",https://www.nytimes.com/2025/09/16/us/politics...
3,2025-09-16-16-45-52 +0000,bbc,UK-US steel tariff agreement put on hold,https://www.bbc.com/news/articles/cj4y2gge7p1o...
4,2025-09-16-16-45-48 +0000,nyt,Sotheby’s Lands a Leonard Lauder Art Collectio...,https://www.nytimes.com/2025/09/15/arts/design...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2287/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
213,trump,55
45,kirk,34
127,charlie,27
66,gaza,22
99,new,21


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
46,2025-09-16-15-56-29 +0000,nypost,Eric Trump says prez saw Charlie Kirk as ‘seco...,https://nypost.com/2025/09/16/us-news/eric-tru...,137
77,2025-09-16-14-44-00 +0000,wsj,FBI Director Kash Patel is defending his work ...,https://www.wsj.com/politics/policy/fbi-direct...,126
252,2025-09-16-00-41-00 +0000,wsj,President Trump got a win when the Senate conf...,https://www.wsj.com/economy/central-banking/st...,122
218,2025-09-16-05-13-50 +0000,nypost,Trump to file $15 billion defamation and libel...,https://nypost.com/2025/09/16/us-news/trump-to...,117
198,2025-09-16-08-49-51 +0000,cbc,Trump launches $15B US defamation lawsuit agai...,https://www.cbc.ca/news/world/trump-lawsuit-ne...,117


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
46,137,2025-09-16-15-56-29 +0000,nypost,Eric Trump says prez saw Charlie Kirk as ‘seco...,https://nypost.com/2025/09/16/us-news/eric-tru...
86,90,2025-09-16-14-32-00 +0000,wsj,Israel Launches New Ground Offensive Seeking t...,https://www.wsj.com/world/middle-east/israel-l...
252,67,2025-09-16-00-41-00 +0000,wsj,President Trump got a win when the Senate conf...,https://www.wsj.com/economy/central-banking/st...
77,65,2025-09-16-14-44-00 +0000,wsj,FBI Director Kash Patel is defending his work ...,https://www.wsj.com/politics/policy/fbi-direct...
120,61,2025-09-16-13-19-00 +0000,wsj,A federal appeals court on Monday night reject...,https://www.wsj.com/economy/central-banking/ap...
80,60,2025-09-16-14-43-00 +0000,wsj,Judge Throws Out Terror-Related Murder Charges...,https://www.wsj.com/us-news/law/judge-throws-o...
118,44,2025-09-16-13-19-35 +0000,nypost,DA Fani Willis loses bid to appeal Trump’s Geo...,https://nypost.com/2025/09/16/us-news/da-fani-...
7,43,2025-09-16-16-45-03 +0000,nyt,Live Updates: Israeli Troops Push Into Gaza Ci...,https://www.nytimes.com/live/2025/09/16/world/...
279,36,2025-09-15-22-18-14 +0000,nypost,"US, China near TikTok deal to finally resolve ...",https://nypost.com/2025/09/15/business/us-chin...
299,34,2025-09-15-20-50-24 +0000,cbc,Trump says U.S. has struck another alleged dru...,https://www.cbc.ca/news/world/us-drug-boat-ven...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
